In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import altair as alt

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        os.path.join(dirname, filename)

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# IF you like this notebook then please give an upvote 👍
**and also give suggestions in comment 🙌**

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import datetime
import tensorflow as tf
import itertools
import matplotlib
from matplotlib import pyplot
import matplotlib.pyplot as plt
from sklearn.utils import class_weight
import tensorflow_addons as tfa
from sklearn.model_selection import StratifiedKFold
import math
from hyperopt import STATUS_OK, Trials, fmin, hp, tpe
import plotly.express as px
from plotly.subplots import make_subplots
import seaborn as sns
import plotly.graph_objects as go
import warnings
warnings.filterwarnings("ignore")
from IPython.display import clear_output
from sklearn.metrics import log_loss
import xgboost as xgb
import random
import optuna
from sklearn.model_selection import KFold, train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import GradientBoostingClassifier
from lightgbm import LGBMClassifier
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split,StratifiedKFold
from catboost import CatBoostClassifier

In [ ]:
df=pd.read_csv('../input/titanic/train.csv')
test_df=pd.read_csv('../input/titanic/test.csv')

In [ ]:
titles1=[]
titles2=[]
marital1=[]
marital2=[]
for i in df['Name']:
    titles1.append(i.split(',')[0])
    marital1.append(i.split('.')[0].split(',')[1])
for i in test_df['Name']:
    titles2.append(i.split(',')[0])
    marital2.append(i.split('.')[0].split(',')[1])

In [ ]:
df['Titles']=titles1
df['marital_status']=marital1
test_df['Titles']=titles2
test_df['marital_status']=marital2
a=df['marital_status'].value_counts()
b=test_df['marital_status'].value_counts()
cleanup_nums = {"marital_status":     {" Mr": 1, " Mrs":1, " Lady": 1, " Miss":0, " Master":0}
               }
df = df.replace(cleanup_nums)
test_df = test_df.replace(cleanup_nums)
df.loc[(df.marital_status.isin([" Dr", " Rev", " Mlle", " Major", " Col", " Mme", " Don", " Capt", " Jonkheer", " Ms", " the Countess", " Sir"])) & (df['SibSp']>0) , 'marital_status'] = 1
df.loc[(df.marital_status.isin([" Dr", " Rev", " Mlle", " Major", " Col", " Mme", " Don", " Capt", " Jonkheer", " Ms", " the Countess", " Sir"])) & (df['SibSp']==0) , 'marital_status'] = 0
test_df.loc[(test_df.marital_status.isin([" Dr", " Rev", " Mlle", " Major", " Col", " Mme", " Don", " Capt", " Jonkheer", " Ms", " the Countess", " Sir", " Dona"])) & (test_df['SibSp']>0) , 'marital_status'] = 1
test_df.loc[(test_df.marital_status.isin([" Dr", " Rev", " Mlle", " Major", " Col", " Mme", " Don", " Capt", " Jonkheer", " Ms", " the Countess", " Sir", " Dona"])) & (test_df['SibSp']==0) , 'marital_status'] = 0

In [ ]:
test_df['marital_status'].unique()

In [ ]:
df['marital_status']=df['marital_status'].astype(int)
test_df['marital_status']=test_df['marital_status'].astype(int)

In [ ]:
df['marital_status'].unique()

In [ ]:
df.head()

In [ ]:
%%capture
df['survived']=df['Sex']
mask=(df['Survived']==1)
df['survived'][mask]='survived'
mask=(df['Survived']==0)
df['survived'][mask]='died'

In [ ]:
df['relatives']=df['SibSp']+df['Parch']

In [ ]:
import missingno as msno

msno.bar(df)
plt.show()

In [ ]:
plt.figure(figsize = (12, 7))

sns.countplot(x = 'survived', data = df, hue='Sex')
plt.show()

**Clearly female survived more**

In [ ]:

fig=px.scatter(df.loc[df['Sex'] == 'male'],
                 x="PassengerId", y="Age", color="Survived",
                 log_x=True, size_max=20,
                 template='plotly', title="Which Ages of males Survived?",opacity=1)
fig.show()      

In [ ]:
fig=px.scatter(df.loc[df['Sex'] == 'female'],
                 x="PassengerId", y="Age", color="Survived",
                 log_x=True, size_max=20,
                 template='plotly', title="Which Age of females Survived?", opacity=1)
fig.show() 

In [ ]:
fig, ax=plt.subplots(figsize=(9,3))
sns.histplot(df['Age'],kde=True)
ax.set_title('Which age-group was more on the ship')
plt.show()

In [ ]:
sns.catplot(x = 'Parch', y = 'Survived', data = df, kind = 'bar', col = 'Pclass', hue='Sex')

In [ ]:
sns.barplot(x='Embarked', y='Survived', data=df);

In [ ]:
sns.barplot(x='relatives', y='Survived', data=df);

In [ ]:
sns.kdeplot(x='SibSp', y='Survived', data=df, shade=True)

In [ ]:
sns.pairplot(data=df, diag_kind='kde', hue='Sex')

In [ ]:
df['married']=df['Sex']
mask=(df['marital_status']==1)
df['married'][mask]='married'
mask=(df['marital_status']==0)
df['married'][mask]='unmarried'
df['use']=df['married']+' '+df['Sex']+' '+df['survived']
fig=px.scatter(df,
                 x="PassengerId", y="Age", color="use",
                 log_x=True, size_max=20,
                 template='plotly', title="which age married vs unmarried who survived?", opacity=1)
df=df.drop(['use','married'], axis=1)
fig.show()

In [ ]:
corrmatrix=df.corr()
fig, ax=plt.subplots(figsize=(10,10))
sns.heatmap(corrmatrix, vmax=8, square=True, annot=True, fmt='.2f')
plt.show()

**data is not highly correlated**

In [ ]:
#sns.catplot(x = 'Sex', y = 'Age', kind = 'box', data = df, height = 5, aspect = 2,col='survived',contor='h')
#plt.show()
fig, ax=plt.subplots(figsize=(15,5))
ax.tick_params(axis='x', rotation=90)
sns.boxplot(data=df, orient='h')

**We can see the Age and Fare columns has very high variance**

In [ ]:
fig, ax=plt.subplots(figsize=(25,10))
sns.histplot(df['Fare'],kde=True)
ax.set_title('Distribution of Fare for passengers')
plt.show()

In [ ]:
#result dataframe
result=pd.DataFrame(df['Sex'])
result['model']=df['Sex']
result['accuracy']=df['Survived']
result=result.drop('Sex', axis=1)
result=result.iloc[len(result):,:]

In [ ]:
#dropping unnecesary columns NOTE survived was a column created to aid visualization
df=df.drop(['PassengerId', 'Name', 'survived', 'Ticket','relatives'], axis=1)
#filling Embarked with most frequent in the column
df['Embarked'] = df['Embarked'].fillna(df['Embarked'].value_counts().index[0])
#replacing manually the categorical variable other encoders could be used
cleanup_nums = {"Embarked":     {"Q": 1, "C":2, "S":3},
                "Sex": {"male": 0, "female": 1}
               }
df = df.replace(cleanup_nums)
#total no of relatives for a person on the ship
df['family']=df['SibSp']+df['Parch']+1

#its not realistic for fare to be zero which is special case so let us replace with mean
df['Fare'] = df['Fare'].replace(0, df['Fare'].mean())
#if has a cabin then 1 else 0
df['Cabin']=df['Cabin'].fillna("0")
df.loc[df.Cabin != "0", 'Cabin'] = "1"
df['Cabin']=df['Cabin'].astype(int)

#if has no relative then 0 else 1
df['isAlone']=np.zeros(len(df))
df.loc[df.family == 1, 'isAlone'] = 1
dp=df.dropna()
#we will not use survived as a feature to predict the age to fill the missing ages as it will not be available in test dataset
X=dp.drop(['Age', 'Titles','Survived'], axis=1)
y=dp.Age

In [ ]:
%%capture
test_df=test_df.drop(['PassengerId', 'Name', 'Ticket'], axis=1)

test_df['family']=test_df['SibSp']+test_df['Parch']+1

test_df['Embarked']=test_df['Embarked'].fillna('G')
test_df = test_df.replace(cleanup_nums)
test_df['Fare'] = test_df['Fare'].replace(0, test_df['Fare'].mean())
test_df['Fare']=test_df['Fare'].fillna(test_df['Fare'].mean())
test_df['Cabin']=test_df['Cabin'].fillna("0")
test_df.loc[df.Cabin != "0", 'Cabin'] = "1"
test_df['Cabin']=test_df['Cabin'].astype(int)
test_df['isAlone']=np.zeros(len(test_df))
test_df.loc[test_df.family == 1, 'isAlone'] = 1
dp=test_df.dropna()
x=dp.drop(['Age','Titles'], axis=1)
Y=dp.Age

In [ ]:
X=pd.concat([X, x])
y=pd.concat([y,Y])
X.reset_index()
y.reset_index()

**Let us see how our data looks now**

In [ ]:
df.head()

In [ ]:
X_train, X_validation, y_train, y_validation = train_test_split(X, y, test_size=0.15)
import xgboost as xgb
from xgboost import XGBRegressor, XGBClassifier

In [ ]:
def checking(train_X, train_y, val_X, val_y, min_child_weight, n_estimators, max_depth):
    score=1000000
    n_emi=0
    min_chi=0
    model=XGBRegressor()
    max_de=0
    for i in n_estimators:
        for j in min_child_weight:
            for k in max_depth:
                xgb_m = XGBRegressor(min_child_weight=j, n_estimators=i, max_depth=k, random_state=0, tree_method='gpu_hist', gpu_id=0)
                xgb_m.fit(train_X,train_y)
                preds=xgb_m.predict(val_X)
                r=mean_squared_error(val_y, preds, multioutput='raw_values')
                r=math.sqrt(r)
                if(r<score):
                    score=r
                    n_emi=i
                    min_chi=j
                    model=xgb_m
                    max_de=k
                print("n_estimators = "+str(i)+" and min_child_weight = "+str(j)+" and max_detph = "+str(k)+" gives accuracy of "+str(r))
    return [model,score, n_emi, min_chi]  

In [ ]:
min_child_weight=[1,2,3,4,5]
max_depth=[2,3,4,5,6,7]
n_estimators=[50,70,60,80,100,120,140,160,180,200,300]
xt=checking(X_train, y_train, X_validation, y_validation, min_child_weight, n_estimators, max_depth)
print(xt)

In [ ]:
df.head()

In [ ]:
%%capture
#now we predict age for the entire train dataset
preds=xt[0].predict(df.drop(['Age','Survived','Titles'], axis=1))
df['Age_d']=preds
#we replace fill only the empty spots by the predicted ages array
df['Age']=df['Age'].fillna(df['Age_d'])
df=df.drop('Age_d', axis=1)
df['Age'][df['Age'] <= 1] = 1

In [ ]:
test_df.head()

In [ ]:
%%capture
#now we predict age for the entire train dataset
preds=xt[0].predict(test_df.drop(['Age','Titles'], axis=1))
test_df['Age_d']=preds
#we replace fill only the empty spots by the predicted ages array
test_df['Age']=test_df['Age'].fillna(test_df['Age_d'])
test_df=test_df.drop('Age_d', axis=1)
test_df['Age'][test_df['Age'] <= 1] = 1

In [ ]:
title=titles1+titles2
res = []
[res.append(x) for x in title if x not in res]
family=pd.DataFrame(res,columns =['Names'])
below_10_male=[]
below_20_male=[]
below_25_male=[]
below_10_female=[]
below_20_female=[]
below_25_female=[]
below_50_female=[]
below_50_male=[]
above_50_female=[]
above_50_male=[]

for i in res:
    below_10_male.append(len(df[((df['Titles']==i) | (test_df['Titles']==i)) & (df['Age']<=10) & (df['Sex']==1)]))
    below_10_female.append(len(df[((df['Titles']==i) | (test_df['Titles']==i)) & (df['Age']<=10) & (df['Sex']==0)]))
    below_20_male.append(len(df[((df['Titles']==i) | (test_df['Titles']==i)) & (df['Age']<=20) & (df['Sex']==0) & (df['Age']>10)]))
    below_20_female.append(len(df[((df['Titles']==i) | (test_df['Titles']==i)) & (df['Age']<=20) & (df['Sex']==1) & (df['Age']>10)]))
    below_25_male.append(len(df[((df['Titles']==i) | (test_df['Titles']==i)) & (df['Age']<=25) & (df['Sex']==0) & (df['Age']>20)]))
    below_25_female.append(len(df[((df['Titles']==i) | (test_df['Titles']==i)) & (df['Age']<=25) & (df['Sex']==1) & (df['Age']>20)]))
    below_50_male.append(len(df[((df['Titles']==i) | (test_df['Titles']==i)) & (df['Age']<=50) & (df['Sex']==0) & (df['Age']>25)]))
    below_50_female.append(len(df[((df['Titles']==i) | (test_df['Titles']==i)) & (df['Age']<=50) & (df['Sex']==1) & (df['Age']>25)]))
    above_50_male.append(len(df[((df['Titles']==i) | (test_df['Titles']==i))  & (df['Sex']==0) & (df['Age']>50)]))
    above_50_female.append(len(df[((df['Titles']==i) | (test_df['Titles']==i))  & (df['Sex']==1) & (df['Age']>50)]))

In [ ]:
family['b_10_m']=below_10_male
family['b_10_fm']=below_10_female
family['b_20_m']=below_20_male
family['b_20_fm']=below_20_female
family['b_25_m']=below_25_male
family['b_25_fm']=below_25_female
family['b_50_m']=below_50_male
family['b_50_fm']=below_50_female
family['a_50_m']=above_50_male
family['a_50_fm']=above_50_female
family.head()

In [ ]:
df['b_10_m']=np.zeros(len(df))
df['b_10_fm']=np.zeros(len(df))
df['b_20_m']=np.zeros(len(df))
df['b_20_fm']=np.zeros(len(df))
df['b_25_m']=np.zeros(len(df))
df['b_25_fm']=np.zeros(len(df))
df['b_50_m']=np.zeros(len(df))
df['b_50_fm']=np.zeros(len(df))
df['a_50_m']=np.zeros(len(df))
df['a_50_fm']=np.zeros(len(df))

test_df['b_10_m']=np.zeros(len(test_df))
test_df['b_10_fm']=np.zeros(len(test_df))
test_df['b_20_m']=np.zeros(len(test_df))
test_df['b_20_fm']=np.zeros(len(test_df))
test_df['b_25_m']=np.zeros(len(test_df))
test_df['b_25_fm']=np.zeros(len(test_df))
test_df['b_50_m']=np.zeros(len(test_df))
test_df['b_50_fm']=np.zeros(len(test_df))
test_df['a_50_m']=np.zeros(len(test_df))
test_df['a_50_fm']=np.zeros(len(test_df))

In [ ]:
for i in res:
    dp=family.loc[family['Names']==i]
    df.loc[df.Titles==i, 'b_10_m']=dp['b_10_m']
    df.loc[df.Titles==i, 'b_20_m']=dp['b_20_m']
    df.loc[df.Titles==i, 'b_25_m']=dp['b_25_m']
    df.loc[df.Titles==i, 'b_50_m']=dp['b_50_m']
    df.loc[df.Titles==i, 'a_50_m']=dp['a_50_m']
    df.loc[df.Titles==i, 'b_10_fm']=dp['b_10_fm']
    df.loc[df.Titles==i, 'b_10_fm']=dp['b_10_fm']
    df.loc[df.Titles==i, 'b_20_fm']=dp['b_20_fm']
    df.loc[df.Titles==i, 'b_25_fm']=dp['b_25_fm']
    df.loc[df.Titles==i, 'b_50_fm']=dp['b_50_fm']
    df.loc[df.Titles==i, 'a_50_m']=dp['a_50_fm']
    test_df.loc[df.Titles==i, 'b_10_m']=dp['b_10_m']
    test_df.loc[df.Titles==i, 'b_20_m']=dp['b_20_m']
    test_df.loc[df.Titles==i, 'b_25_m']=dp['b_25_m']
    test_df.loc[df.Titles==i, 'b_50_m']=dp['b_50_m']
    test_df.loc[df.Titles==i, 'a_50_m']=dp['a_50_m']
    test_df.loc[df.Titles==i, 'b_10_fm']=dp['b_10_fm']
    test_df.loc[df.Titles==i, 'b_10_fm']=dp['b_10_fm']
    test_df.loc[df.Titles==i, 'b_20_fm']=dp['b_20_fm']
    test_df.loc[df.Titles==i, 'b_25_fm']=dp['b_25_fm']
    test_df.loc[df.Titles==i, 'b_50_fm']=dp['b_50_fm']
    test_df.loc[df.Titles==i, 'a_50_m']=dp['a_50_fm']

In [ ]:
df=df.fillna(0)
test_df=test_df.fillna(0)

In [ ]:
df=df.drop('Titles', axis=1)
test_df=test_df.drop('Titles', axis=1)
df.head()

In [ ]:
corrmatrix=df.corr()
plt.figure(figsize=(20,20))
plt.title('Let us Again see the final correlation')
sns.heatmap(corrmatrix,vmax=2.0, 
            square=True, annot=True)

**From this it is clear that children being with or without parents(with Nanny) doesn't have strong influence on survival**

In [ ]:
# the following two columns have high variance
df['Age'] = np.log(df['Age'])
df['Fare'] = np.log(df['Fare'])
X=df.drop('Survived', axis=1)
y=df.Survived
#split for training and validation
X_train, X_validation, y_train, y_validation = train_test_split(X, y, test_size=0.15)

In [ ]:
accu=[]
models=[]

# XGBoost

In [ ]:
def checking(train_X, train_y, val_X, val_y, min_child_weight, n_estimators, max_depth):
    score=0
    n_emi=0
    min_chi=0
    model=XGBClassifier()
    max_de=0
    for i in n_estimators:
        for j in min_child_weight:
            for k in max_depth:
                xgb_m = XGBClassifier(min_child_weight=j, n_estimators=i, max_depth=k, random_state=0, tree_method='gpu_hist', gpu_id=0)
                xgb_m.fit(train_X,train_y)
                acc=xgb_m.score(X_validation,y_validation)
                if(acc>score):
                    score=acc
                    n_emi=i
                    min_chi=j
                    model=xgb_m
                    max_de=k
                print("n_estimators = "+str(i)+" and min_child_weight = "+str(j)+" and max_detph = "+str(k)+" gives accuracy of "+str(acc))
    return [model,score, n_emi, min_chi] 

In [ ]:
min_child_weight=[1,2,3,4,5,6]
max_depth=[2,3,4,5,6,7,8]
n_estimators=[50,70,60,80,100,120,140,160,180,200,300]
xt=checking(X_train, y_train, X_validation, y_validation, min_child_weight, n_estimators, max_depth)
print(xt)

In [ ]:
pyplot.bar(range(len(xt[0].feature_importances_)), xt[0].feature_importances_)
pyplot.show()

# Random Forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier
def checking(train_X, train_y, val_X, val_y, n_estimators, max_depth):
    score=0
    n_emi=0
    model=RandomForestClassifier()
    max_de=0
    for i in n_estimators:
        for k in max_depth:
            xgb_m = RandomForestClassifier(n_estimators=i, max_depth=k, random_state=0)
            xgb_m.fit(train_X,train_y)
            acc=xgb_m.score(X_validation,y_validation)
            if(acc>score):
                score=acc
                n_emi=i
                model=xgb_m
                max_de=k
            print("n_estimators = "+str(i)+" and max_detph = "+str(k)+" gives accuracy of "+str(acc))
    return [model,score, n_emi, max_de] 

In [ ]:
max_depth=[2,3,4,5,6,7]
n_estimators=[50,70,60,80,100,120,140,160,180,200,300]
rt=checking(X_train, y_train, X_validation, y_validation, n_estimators, max_depth)
print(rt)

In [ ]:
pyplot.bar(range(len(rt[0].feature_importances_)), rt[0].feature_importances_)
pyplot.show()

# if you like this then please upvote 👍

In [ ]:
t_df=pd.read_csv('../input/titanic/gender_submission.csv')
preds=rt[0].predict(test_df)
t_df['Survived']=preds
t_df['Survived']=t_df['Survived'].round()
t_df['Survived']=t_df['Survived'].astype(int)
import gzip
t_df.to_csv('sub.gz', index=False, compression='gzip')
t_df.head(10)

In [ ]:
t_df=pd.read_csv('../input/titanic/gender_submission.csv')
preds=xt[0].predict(test_df)
t_df['Survived']=preds
t_df['Survived']=t_df['Survived'].round()
t_df['Survived']=t_df['Survived'].astype(int)
import gzip
t_df.to_csv('subm.gz', index=False, compression='gzip')
t_df.head(10)